# SpikeInterface Tutorial - Spike Sorting Workshop - Edinburgh 2019


In this tutorial, we will cover the basics of using SpikeInterface for extracellular analysis and spike sorting comparison. We will be using three packages from the SpikeInterface github organization: spikeextractors, spiketoolkit, and spikewidgets.

For this analysis, we will be using a simulated dataset from [MEArec](https://github.com/alejoe91/MEArec). We will show how to:

- load the data with spikeextractors package
- load a probe file
- preprocess the signals
- run a popular spike sorting algorithm with different parameters
- curate the spike sorting output using Phy
- compare with ground-truth information
- run consensus-based spike sorting


For this tutorial we will need the following packages:
- spikeextractors
- spiketoolkit
- spikewidgets
- MEArec
- klusta
- phy
- matplotlib

+ all their dependencies.

To install those you can use the `requirements.txt` in this directory by running the command:

`pip install -r requirements.txt`

If you use a conda environment, you can create the `spiketutorial` environment with:

`conda env create -f environment.yml`

You might need to run:

`ipython kernel install --user --name=tutorial`

or:

`conda install nb_conda_kernels` and change Kernel to run the tutorial notebook.

### Downloading the recording

First, we need to download the recording. Feel free to use your own recordings as well later on.

From this Zenodo [link](https://zenodo.org/record/3256071#.XRHqhnX7Q5k), you can download the simulated dataset mentioned above. Move the dataset in the current folder.

The recording was generated on a shank probe with 4 tetrodes separated by 300 $\mu$m. It has 36 cells in total, distributed in the proximity of the 4 tetrodes. The recording is 30 s long and there is an additive noise level of 10 $\mu$V.

In [ ]:
%matplotlib notebook
import spikeextractors as se 
import spiketoolkit as st
import spikewidgets as sw
import matplotlib.pylab as plt
import numpy as np

### Loading recording and probe information

In [ ]:
recording_file = 'recordings_36cells_four-tetrodes_30.0_10.0uV_20-06-2019_14_48.h5'
recording = se.MEArecRecordingExtractor(recording_file, locs_2d=True)

A `RecordingExtractor` object extracts information about channel ids, channel locations (if present), the sampling frequency of the recording, and the extracellular traces (when prompted). The MEArecRecordingExtractor is designed specifically for MEArec datasets.

Here we load information from the recording using the built-in functions from the RecordingExtractor

In [ ]:
channel_ids = recording.get_channel_ids()
fs = recording.get_sampling_frequency()
num_chan = recording.get_num_channels()

print('Channel ids:', channel_ids)
print('Sampling frequency:', fs)
print('Number of channels:', num_chan)

Let's plot the channel locations and a snippet of traces using `spikewidgets`:

In [ ]:
sw.plot_electrode_geometry(recording, elec_size=2)

The `get_traces()` function returns a NxT numpy array where N is the number of channel ids passed in (all channel ids are passed in by default) and T is the number of frames (determined by start_frame and end_frame).

In [ ]:
trace_snippet = recording.get_traces(start_frame=int(fs*0), end_frame=int(fs*2))

In [ ]:
print('Traces shape:', trace_snippet.shape)

In [ ]:
sw.plot_timeseries(recording)

We can see that the spikes mainly appear separately on different tetrodes. Each tetrode belongs to a different `group`. We can load the `group` information in two ways:

- using the `set_channel_groups` in your RecordingExtractor (manually loading group information)
- loading a probe file using the `load_probe_file` from `spikeextractors` (automatically loading group information)

Let's use the second option. Probe files (`.prb`) also enable users to change the channel map (reorder the channels) and add channel grouping properties and locations. In this case, our probe file will order the channels in reverse and split them in 4 groups, representing the 4 tetrodes.

In [ ]:
!cat tetrode_16.prb

In [ ]:
recording_prb = se.load_probe_file(recording, 'tetrode_16.prb')

In [ ]:
print('Original channels:', recording.get_channel_ids())
print('Channels after loading the probe file:', recording_prb.get_channel_ids())
print('Channel groups after loading the probe file:', recording_prb.get_channel_groups())

### Preprocessing recordings


Now that the probe information is loaded we can do some preprocessing using `spiketoolkit`.

We can filter the recordings, rereference the signals to remove noise, discard noisy channels, whiten the data, remove stimulation artifacts, etc. (more info [here](https://spiketoolkit.readthedocs.io/en/latest/preprocessing_example.html)).

For this notebook, let's filter the recordings, remove a noisy channel, and apply common median reference (CMR). All preprocessing modules return new `RecordingExtractor` objects that apply the underlying preprocessing function. This allows users to access the preprocessed data in the same way as the raw data.

Below, we bandpass filter the recording, remove channel 5, and apply common median reference to the original recording.

In [ ]:
recording_f = st.preprocessing.bandpass_filter(recording_prb, freq_min=300, freq_max=6000)
recording_rm_noise = st.preprocessing.remove_bad_channels(recording_f, bad_channels=[5])
recording_cmr = st.preprocessing.common_reference(recording_rm_noise, reference='median')

Now we can extractor traces from the preprocessed recording.

In [ ]:
trace_f_snippet = recording_f.get_traces(start_frame=int(fs*0), end_frame=int(fs*2))
trace_cmr_snippet = recording_cmr.get_traces(start_frame=int(fs*0), end_frame=int(fs*2))

print(trace_f_snippet.shape)
print(trace_cmr_snippet.shape)

We can plot the bandpassfiltered snippets below

In [ ]:
sw.plot_timeseries(recording_f, channels=range(16))

In [ ]:
print('Channel ids for CMR recordings:', recording_cmr.get_channel_ids())
print('Channel groups for CMR recoridng:', recording_cmr.get_channel_groups())

### Spike sorting

We can now run spike sorting on the above recording. We will use `klusta` for this demonstration and we will run spike sorting on each group separately.

Let's first check the installed sorters in spiketoolkit to see if klusta is available. Then we can check the `klusta` default parameters.

We will sort the bandpass filtered recording (the `recording_bpf` object), as there is no external noise and all channels are good :)

In [ ]:
st.sorters.installed_sorter_list

In [ ]:
st.sorters.KlustaSorter.default_params()

We will set the `adjacency_radius` to 50 microns as electrodes belonging to the same tetrode are within this distance.

In [ ]:
# run spike sorting on entire recording
sorting_KL_all = st.sorters.run_klusta(recording_f, output_folder='results_all_klusta', adjacency_radius=50)
print('Found', len(sorting_KL_all.get_unit_ids()), 'units')

In [ ]:
# run spike sorting by group
sorting_KL_split = st.sorters.run_klusta(recording_f, adjacency_radius=50, 
                                         output_folder='results_split_klusta', grouping_property='group')
print('Found', len(sorting_KL_split.get_unit_ids()), 'units')

The spike sorting returns a `SortingExtractor` object. Let's see some of its functions:

In [ ]:
print('Units', sorting_KL_split.get_unit_ids())

In [ ]:
print('Units', sorting_KL_split.get_unit_spike_train(0))

We can use `spikewidgets` functions to quickly visualize some unit features:

In [ ]:
sw.plot_unit_waveforms(sorting=sorting_KL_split, recording=recording_f, unit_ids=range(5))

In [ ]:
sw.plot_rasters(sorting_KL_split)

### Manual curation

To perform manual curation we will export the data to Phy. 

In [ ]:
st.postprocessing.export_to_phy(recording_f, sorting_KL_all, output_folder='phy_KL_all', grouping_property='group')
st.postprocessing.export_to_phy(recording_f, sorting_KL_split, output_folder='phy_KL_split', grouping_property='group')

In [ ]:
%%capture --no-display
!phy template-gui phy_KL_split/params.py

In [ ]:
%%capture --no-display
!phy template-gui phy_all/params.py

After curating the results we can reload it using the `PhySortingExtractor`:

In [ ]:
sorting_KL_all_curated = se.PhySortingExtractor('phy_all/')
sorting_KL_split_curated = se.PhySortingExtractor('phy_split/')

### Some more spike sorting!

If you have other sorters installed, you can try to run them:

In [ ]:
%%capture --no-display
sorting_MS4 = st.sorters.run_mountainsort4(recording_f, grouping_property='group')

In [ ]:
len(sorting_MS4.get_unit_ids())

In [ ]:
st.postprocessing.export_to_phy(recording_f, sorting_MS4, output_folder='phy_MS4', grouping_property='group')

In [ ]:
%%capture --no-display
!phy template-gui  phy_MS4/params.py

In [ ]:
sorting_MS4_curated = se.PhySortingExtractor('phy_MS4')

### Comparison with ground-truth

MEArec recordings are simulated, therefore we know ground truth information about the spiking times. 
We can load the ground truth `SortingExtractor` as:

In [ ]:
sorting_gt = se.MEArecSortingExtractor(recording_file)

Now we can compare the sorting output to the ground truth information:

In [ ]:
cmp_KL = st.comparison.compare_sorter_to_ground_truth(sorting_gt, sorting_KL_split, min_accuracy=0.5)

In [ ]:
cmp_KL.get_performance()

In [ ]:
cmp_KL.get_performance(method='pooled_with_average')

In [ ]:
cmp_MS4 = st.comparison.compare_sorter_to_ground_truth(sorting_gt, sorting_MS4, min_accuracy=0.5)

In [ ]:
cmp_MS4.get_performance()

In [ ]:
cmp_MS4.get_performance(method='pooled_with_average')

## Exercise) Can you improve the performance with manual curation?

### Multi-sorting comparison

Finally, we can compare KL and SC (or more) and automatically curate the sorting output by retaining the matching units between the two (or more) sorters. We will use the `compare_multiple_sorters` function.
The multi sorting comparison builds a graph with all the units from the different sorters, connected with their agreement score. We can use this to extract agreement sorting.

In [ ]:
msc = st.comparison.compare_multiple_sorters(sorting_list=[sorting_KL_split, sorting_MS4], name_list=['KL', 'MS4'])

In [ ]:
sorting_agreement = msc.get_agreement_sorting(minimum_matching=2)

In [ ]:
print('Klusta units', len(sorting_KL_split.get_unit_ids()))
print('Mountainsort units', len(sorting_MS4.get_unit_ids()))
print('Agreement units', len(sorting_agreement.get_unit_ids()))

We can still inspect the agreement sorting using Phy:

In [ ]:
st.postprocessing.export_to_phy(recording_f, sorting_agreement, output_folder='phy_AGR', grouping_property='group')

In [ ]:
%%capture --no-display
!phy template-gui phy_AGR/params.py